In [60]:
import glob
import os
import csv
import numpy as np
    
def information_gain(angles,bank_particles,weights):
    """
    Function is now vectorized in 0th index.
    Takes in angles as [theta,phi] and bank_particles and weigt as how they are given by the file. 
    """
    best_guess=np.array(np.einsum('ijkl,ij->ikl',bank_particles,weights))
    return adaptive_cost_func(angles,bank_particles,weights,best_guess)       

def adaptive_cost_func(angles,rhoBank,weights,bestGuess):
    """
    Function is now vectorized in 0th index.
    Computes the expected entropy reduction of the posterior (likelihood) distribution.
    The angles are taken in as a dictionary and indicate what mesaurement is to be perfomred.
    Noise correction is currently removed.  
    """
    # Crates projector from angles
    povm=angles_to_single_qubit_POVM(angles)
    # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
    K=Shannon_entropy(np.einsum('nijk,nkj->ni',povm,bestGuess))
    J=Shannon_entropy(np.einsum('nijk,nlkj->nil',povm,rhoBank))
    # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
    return np.real(K-np.einsum("ij,ij->i",J,weights))

def Shannon_entropy(prob):
    """
    Function is now vectorized in 0th index.
    Returns the shannon entorpy of the probability histogram. 
    """
    return np.real(np.sum(-(prob*np.log2(prob)),axis=1))

def angles_to_single_qubit_POVM(angles):
    """
    Function is now vectorized in 0th index.
    Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array . 
    For single qubit only.
    """
    up_state_vector=np.array([np.cos(angles[:,0]/2),np.exp(1j*angles[:,1])*np.sin(angles[:,0]/2)],dtype=complex)
    up_POVM=np.einsum("in,jn->nij",up_state_vector,up_state_vector.conj())
    return np.einsum('injk->nijk',np.array([up_POVM[:],np.eye(2)-up_POVM[:]],dtype=complex))



path= "SL_data/"
cvs_names=glob.glob(f'{path}*.csv')
bank_names=glob.glob(f'{path}*bank.npy')
weight_names=glob.glob(f'{path}*weights.npy')

        
NN_angle_pred=np.array([1,0.5]) # Format as [ theta, phi ]

angles_list   = []
banks_list   = []
weights_list = []
for id, name in enumerate(cvs_names):
    data    = np.loadtxt( cvs_names[id],delimiter=',',skiprows=1)
    bank    = np.load(bank_names[id],mmap_mode="r")[1:]
    weights = np.load(weight_names[id],mmap_mode="r")[1:]
    angles_list.append(data)
    banks_list.append(bank)
    weights_list.append(weights)

#print(angles_list)
angles_data   = np.stack(angles_list)
banks_data    = np.stack(banks_list)
weights_data  = np.stack(weights_list)

print(angles_data.shape)
print(banks_data.shape)
print(weights_data.shape)

# Select which step in ABME to predict
prediction_index=50

true_angle=angles_data[:,prediction_index,3:]
true_info_gain=information_gain(true_angle,banks_data[:,prediction_index],weights_data[:,prediction_index])


# Test to see that step 10 to 60 works
for i in range(50):
    prediction_index=10 +i
    true_angle=angles_data[:,prediction_index,3:]
    true_info_gain_extra=information_gain(true_angle,banks_data[:,prediction_index],weights_data[:,prediction_index])


(1003, 99, 5)
(1003, 99, 100, 2, 2)
(1003, 99, 100)
